## 필요 라이브러리

In [1]:
import copy
import os
import pandas as pd
import numpy as np
# os.chdir('C:/Users/user/Desktop/잡')
from haversine import haversine

# 최적 경로 작업 시작

##### 출발지, 도착지 설정 & 경로 설정에 필요한 데이터 정리

In [2]:
departure = '상계3,4동' #출발지점
destination = '공릉2동' #도착지점
print("-----[",departure," -> ",destination,"]-----")


landscape={
    '상계1동' : {'상계3,4동':0,'상계9동':0,'상계8동':0,'상계5동':0},
    '상계3,4동':{'중계4동':0, '상계5동':0,'상계1동':0},
    '상계5동':{'상계9동':0,'상계1동':0,'상계3,4동':0,'중계4동':0,'상계2동':0},
    '상계9동':{'상계1동':0,'상계5동':0,'상계8동':0,'상계10동':0},
    '상계8동':{'상계1동':0,'상계9동':0,'상계10동':0},
    '상계10동':{'상계8동':0,'상계9동':0,'상계2동':0,'상계6,7동':0},
    '상계2동':{'상계5동':0,'상계10동':0,'상계6,7동':0,'중계4동':0,'중계1동':0},
    '중계4동':{'상계3,4동':0,'상계5동':0,'상계2동':0,'중계1동':0,'중계본동':0},
    '상계6,7동':{'상계10동':0,'상계2동':0,'중계2,3동':0,'월계2동':0,'하계2동':0},
    '중계2,3동':{'상계6,7동':0,'중계1동':0,'하계1동':0,'하계2동':0},
    '중계1동':{'상계2동':0,'중계4동':0,'중계본동':0,'하계1동':0,'중계2,3동':0},
    '중계본동':{'중계4동':0,'중계1동':0,'하계1동':0,'공릉2동':0},
    '하계1동':{'중계1동':0,'중계본동':0,'중계2,3동':0,'하계2동':0,'공릉2동':0},
    '하계2동':{'중계2,3동':0,'상계6,7동':0,'하계1동':0,'월계3동':0,'공릉1동':0},
    '월계2동':{'상계6,7동':0,'월계3동':0,'월계1동':0},
    '월계1동':{'월계2동':0,'월계3동':0},
    '월계3동':{'월계1동':0,'월계2동':0,'공릉1동':0,'하계2동':0},
    '공릉1동':{'월계3동':0,'하계2동':0,'공릉2동':0},
    '공릉2동':{'공릉1동':0,'하계1동':0,'중계본동':0}
    }

# 동별 위도, 경도
Dong_dict = {'월계1동': (37.63019940184156, 127.06595931151917),
 '월계2동': (37.64490995356445, 127.0605321871607),
 '월계3동': (37.63498486169752, 127.07081648041262),
 '공릉2동': (37.64548852468367, 127.11248319287002),
 '하계1동': (37.64630116459204, 127.08373317141721),
 '하계2동': (37.63957602617072, 127.07035648321087),
 '중계본동': (37.65777243662645, 127.09456996970314),
 '중계1동': (37.65517736301124, 127.07817360846009),
 '중계4동': (37.66461204792735, 127.09541063032314),
 '상계1동': (37.69613745772456, 127.08517719487207),
 '상계2동': (37.66156826026975, 127.07202312964317),
 '상계5동': (37.674099521678116, 127.07569521533226),
 '상계8동': (37.67118835605322, 127.05836444631622),
 '상계9동': (37.67273947471108, 127.0685854756317),
 '상계10동': (37.6651921548431, 127.065651391478),
 '상계3,4동': (37.69038984836544, 127.09644725709585),
 '상계6,7동': (37.65574831890184, 127.06877717329392),
 '중계2,3동': (37.65048531321757, 127.07183122777153),
 '공릉1동': (37.63152127844566, 127.08166602486317)
 }


-----[ 상계3,4동  ->  공릉2동 ]-----


##### 최적 경로 계산

In [3]:
land=copy.deepcopy(landscape)
# 동별 거리 계산
for i in land.keys():
    a=Dong_dict[i]
    for j in land[i].keys():
        b=Dong_dict[j]
        dis=haversine(a,b, unit = 'km')
        land[i][j]=dis
        
# a = 'hairshop'

# if a in landscape.keys():
#     for i in landscape[a].keys():
#         landscape[a][i] = 1000
        
# for j in list(landscape.keys()):
#     if a in landscape[j].keys():
#         landscape[j][a] = 1000

# 동별 인구 밀집 단계
data_dir = 'C:/workspace/Bigcon/Data/'
step_data=pd.read_csv(os.path.join(data_dir,'Target_data_04_18.csv'),index_col=0)

step_data.Standard.values[0]
step_data=step_data.reset_index(drop=True)
step_data
step_inter=[]
for i in range(len(step_data)):
    interval=[]
    ch=''
    p=0
    for j in step_data['Standard'][i]:
        if j=="\n":
            break
        if j=='[' or j=='(':
            p=1
            continue
        if j==']' or j==')':
            p=0
            if len(ch)>0:
                interval.append(ch.strip(' '))
                ch=''
        if (p==1) and (j!='[' or j!='(' or j!=' '):
            if j!=',':
                ch+=j
            else:
                interval.append(ch.strip(' '))
                ch=''
    step_inter.append(interval)

for i in range(len(step_inter)):
    step_inter[i]=list(map(float,step_inter[i]))
TMST=list(step_data['TMST_15']) # 시간 조정
dong=list(step_data['HDONG_NM'])

step={}
for i in range(len(TMST)):
    for j in range(len(step_inter[i])):
        if step_inter[i][j]>TMST[i]:
            if j==0:
                step[dong[i]]=1
            else:
                step[dong[i]]=(j+1)//2
            break
        if j==9: # 5단계 : 
            step[dong[i]]=5

# step={'월계1동': 0,
#  '월계2동': 0,
#  '월계3동': 0,
#  '공릉2동': 0,
#  '하계1동': 0,
#  '하계2동': 0,
#  '중계본동': 0,
#  '중계1동': 0,
#  '중계4동': 0,
#  '상계1동': 0,
#  '상계2동': 0,
#  '상계5동': 0,
#  '상계8동': 0,
#  '상계9동': 0,
#  '상계10동': 0,
#  '상계3,4동': 0,
#  '상계6,7동': 0,
#  '중계2,3동': 0,
#  '공릉1동': 0
#  }

routing = {}

for place in land.keys():
    routing[place] = {'sdis':0, 'route':[], 'visited':0}
    
def visitDong(visit):
    routing[visit]['visited'] = 1 #방문
    for go, dis in land[visit].items(): # 거리 재계산
        todist = routing[visit]['sdis'] + dis
        if ((routing[go]['sdis'] >= todist) or len(routing[go]['route'])==0) and step[go]<4:
            routing[go]['sdis'] = todist # 거리가 더 짧은 것이 있다면 다시 넣기
            routing[go]['route'] = copy.deepcopy(routing[visit]['route'])
            routing[go]['route'].append(visit)

visitDong(departure)


while True:
    dis_min = max(routing.values(), key=lambda x : x['sdis'])['sdis'] #최대로 초기화
    visit = False
    for Dong, info in routing.items():
        if 0 < info['sdis'] <= dis_min and info['visited']==0: #방문 안한 곳 중에서 최소거리 동
            dis_min = info['sdis']
            visit = Dong  # 방문
            
    if visit == False: # 방문한 곳이 없으면 종료
        break
    
    visitDong(visit)
    
    print("[" + visit + "]")
    print("Dist : ",dis_min)
    
print("\n", "출발 :", departure,'   ','도착 :', destination)
print("경로 : ", routing[destination]['route'] + [destination])
print("최단거리 : ", routing[destination]['sdis'])


[상계1동]
Dist :  1.1797416620773338
[상계5동]
Dist :  2.5721995699853557
[상계9동]
Dist :  3.215955403843069
[상계2동]
Dist :  4.002608440856764
[상계10동]
Dist :  4.0940163333831565
[상계8동]
Dist :  4.1319283699039495
[상계6,7동]
Dist :  4.710032205920671
[중계2,3동]
Dist :  5.354060001293116
[월계2동]
Dist :  6.116924151839807
[하계1동]
Dist :  6.5005684960327805
[하계2동]
Dist :  6.5136792175338165
[월계1동]
Dist :  7.821042399587374
[공릉1동]
Dist :  7.8530680500257946
[공릉2동]
Dist :  9.03345741878351

 출발 : 상계3,4동     도착 : 공릉2동
경로 :  ['상계3,4동', '상계5동', '상계2동', '상계6,7동', '중계2,3동', '하계1동', '공릉2동']
최단거리 :  9.03345741878351
